# Optimointi pythonilla SciPy:n linprog-funktiolla

In [ ]:
import numpy as np
from scipy.optimize import linprog

# Inputs
Käytämme samaa esimerkkiä kuin aikaisemmin, eli **rakennamme tuoleja ja pöytiä legoilla**. Tätä varten ohjelmaan syötetään erikokoisten legojen määrät ja hinnat, sekä valmiiden huonekalujen myyntihinnat. Nämä tiedot tallennetaan **muuttujiin**.

In [ ]:
# Lego määrät
lego_pieni_määrä = 8
lego_iso_määrä = 6

# Lego hinnat
lego_pieni_hinta = 3
lego_iso_hinta = 5

# Tuotteiden hinnat
tuoli_myyntihinta = 21
pöytä_myyntihinta = 32

# Määritellään funktiot
Määritellään seuraavaksi **kohdefunktio** eli myyntivoiton funktio. Yhteen tuoliin tarvitaan 1 pieni ja 2 isoa legoa, yhteen pöytään tarvitaan 2 pientä ja 2 isoa legoa. x_1 on pienten ja x_2 isojen käytettävissä olevien legojen määrä

Halutaan maksimoida myyntivoitto, eli lisätään ohjelmaan seuraava funktio:
     
$${tuoli\:myynti\:hinta*tuolit+pöytä\:myynti\:hinta*pöydät-lego\:pieni\:hinta*(x_1)-lego\:iso\:hinta*(x_2)}$$

missä
   
$$x_1 = {2*tuolit+2*pöydät}$$
$$x_2 = {1*tuolit+2*pöydät}$$

(tuoli_myyntihinta * tuolit + pöytä_myyntihinta * pöydät - lego_pieni_hinta * (2 * tuolit + 2 * pöydät) - lego_iso_hinta * (tuolit + 2 * pöydät))

In [ ]:
# Kohdefunktio (maksimointi):
# Max (tuoli_myyntihinta * tuolit + pöytä_myyntihinta * pöydät
#      - lego_pieni_hinta * (2 * tuolit + 2 * pöydät)
#      - lego_iso_hinta * (tuolit + 2 * pöydät))
#
# Kerroinmuodossa: Max (c_max · x), missä
# c_max = [
#   tuoli_myyntihinta - 2*lego_pieni_hinta - 1*lego_iso_hinta,
#   pöytä_myyntihinta - 2*lego_pieni_hinta - 2*lego_iso_hinta
# ]

c_max = np.array([
    tuoli_myyntihinta - 2 * lego_pieni_hinta - 1 * lego_iso_hinta,
    pöytä_myyntihinta - 2 * lego_pieni_hinta - 2 * lego_iso_hinta
], dtype=float)

# linprog minimoi, joten minimoidaan -c_max · x
c = -c_max

# Rajoitteet
Tuolien ja pöytien rakentamiseen liittyi myös tiettyjä rajoitteita, jotka pitää ottaa huomioon: palikoita on saatavilla vain 8 pientä palaa ja 6 isoa.

$${2*tuolit+2*pöydät} \le lego\:pieni\:määrä$$
$${1*tuolit+2*pöydät} \le lego\:iso\:määrä$$

In [ ]:
# Pienet legot: 2*tuolit + 2*pöydät <= lego_pieni_määrä
# Isot legot:   1*tuolit + 2*pöydät <= lego_iso_määrä
A_ub = np.array([
    [2.0, 2.0],  # pieni rajoite
    [1.0, 2.0],  # iso rajoite
])
b_ub = np.array([
    float(lego_pieni_määrä),
    float(lego_iso_määrä),
])

# Yhtäsuuruusrajoitteita ei ole
A_eq = None
b_eq = None

# Raja-arvot: tuolit >= 0, pöydät >= 0
bounds = [(0, None), (0, None)]

# Lasketaan lopputulos
Tulostetaan lopputulos, eli optimaalinen määrä tuoleja ja pöytiä. Sen lisäksi voidaan myös tulostaa näiden tuottama myyntivoitto.

In [ ]:
# --------------------------
# Ratkaistaan jatkuva LP (relaxaatio)
# --------------------------
res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq,
              bounds=bounds, method='highs')

# Tulostetaan optimiratkaisu


In [ ]:
if not res.success:
    print("Optimointi epäonnistui:", res.message)
else:
    tuolit_opt, pöydät_opt = res.x
    # Palauta maksimiarvo: obj = -res.fun
    jatkuva_voitto = -res.fun

    # Tulostetaan optimiratkaisu (jatkuva)
    print("Optimiratkaisu (jatkuva, ei-integereitä):")
    print(f"Tuoleja: {tuolit_opt:.6g}")
    print(f"Pöytiä:  {pöydät_opt:.6g}")

    # Lasketaan myös alun perin määritelty myyntivoitto suoraan kaavalla varmistukseksi
    # (pitäisi vastata jatkuva_voitto)
    total_profit_cont = (
        tuoli_myyntihinta * tuolit_opt
        + pöytä_myyntihinta * pöydät_opt
        - lego_pieni_hinta * (2 * tuolit_opt + 2 * pöydät_opt)
        - lego_iso_hinta * (tuolit_opt + 2 * pöydät_opt)
    )
    print(f"\nMyyntivoitto (jatkuva): {total_profit_cont:.2f} €")

    # Jäikö jotain yli? (jatkuva)
    käytetyt_pienet = 2 * tuolit_opt + 2 * pöydät_opt
    käytetyt_isot = tuolit_opt + 2 * pöydät_opt

    lego_pieni_yli = lego_pieni_määrä - käytetyt_pienet
    lego_iso_yli = lego_iso_määrä - käytetyt_isot

    print(f"{lego_pieni_yli:.6g} kpl pieniä legoja jäi yli (jatkuva).")
    print(f"{lego_iso_yli:.6g} kpl isoja legoja jäi yli (jatkuva).")

    myyntihinta_cont = tuoli_myyntihinta * tuolit_opt + pöytä_myyntihinta * pöydät_opt
    print(f"\nMyyntihinta (jatkuva): {myyntihinta_cont:.2f} €")

# --------------------------
# Valinnainen: integer-optimi (brute force pienessä ongelmassa)
# Koska alkuperäinen malli oli kokonaislukuinen, etsitään paras kokonaisratkaisu
# käymällä läpi kaikki järkevät kombinaatiot (pieni hakutila).
# --------------------------
paras_voitto = -np.inf
paras_t = None
paras_p = None

# Ylärajat tiukennetaan rajoitteista: 2t+2p <= 8 -> t+p <= 4, ja t+2p <= 6
# Turvallisesti voidaan iteroida 0..lego_pieni_määrä ja 0..lego_iso_määrä
for t in range(0, lego_pieni_määrä + 1):
    for p in range(0, lego_iso_määrä + 1):
        if (2*t + 2*p) <= lego_pieni_määrä and (t + 2*p) <= lego_iso_määrä:
            # Sama voittofunktio
            voitto = (
                tuoli_myyntihinta * t
                + pöytä_myyntihinta * p
                - lego_pieni_hinta * (2 * t + 2 * p)
                - lego_iso_hinta * (t + 2 * p)
            )
            if voitto > paras_voitto:
                paras_voitto = voitto
                paras_t = t
                paras_p = p

print("\nOptimiratkaisu (kokonaisluvut):")
print(f"Tuoleja: {paras_t}")
print(f"Pöytiä:  {paras_p}")
print(f"\nMyyntivoitto (kokonaisluvut): {paras_voitto:.2f} €")

käytetyt_pienet_int = 2 * paras_t + 2 * paras_p
käytetyt_isot_int = paras_t + 2 * paras_p
lego_pieni_yli_int = lego_pieni_määrä - käytetyt_pienet_int
lego_iso_yli_int = lego_iso_määrä - käytetyt_isot_int

print(f"{lego_pieni_yli_int} kpl pieniä legoja jäi yli (kokonaisluvut).")
print(f"{lego_iso_yli_int} kpl isoja legoja jäi yli (kokonaisluvut).")

myyntihinta_int = tuoli_myyntihinta * paras_t + pöytä_myyntihinta * paras_p
print(f"\nMyyntihinta (kokonaisluvut): {myyntihinta_int:.2f} €")